In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns

import os
import cv2
import random
from skimage.io import imshow
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix



import tensorflow as tf 
from tensorflow import keras


from tensorflow.keras import datasets, layers, models

from tensorflow.keras import optimizers, losses, utils 
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model, Sequential # type of model
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
DATADIR = '/kaggle/input/eurosat/2750/'
CATEGORIES = ["River","Forest","AnnualCrop","HerbaceousVegetation","Highway","Industrial","Pasture","PermanentCrop","Residential","SeaLake"]

training_data = []

def create_image_set():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                training_data.append([img_array,class_num])
            except Exception as e:
                pass
#             plt.imshow(img_array,cmap="gray")
#             plt.show()
#             break
#         break

create_image_set()

In [ ]:
print(training_data)
random.shuffle(training_data)

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
ye=pd.DataFrame(y)
    
New=pd.DataFrame(columns=CATEGORIES)
ye=pd.concat([ye, New],axis=1)
#ye=ye.fillna(0)

ye.loc[ye['River'].isna()==True, ('River','Forest','AnnualCrop','HerbaceousVegetation','Highway','Industrial','Pasture','PermanentCrop','Residential','SeaLake')] = 0.0
ye.loc[ye[0] ==0, 'River'] = 1.0
ye.loc[ye[0] ==1, 'Forest'] = 1.0
ye.loc[ye[0] ==2, 'AnnualCrop'] = 1.0
ye.loc[ye[0] ==3, 'HerbaceousVegetation'] = 1.0
ye.loc[ye[0] ==4, 'Highway'] = 1.0
ye.loc[ye[0] ==5, 'Industrial'] = 1.0
ye.loc[ye[0] ==6, 'Pasture'] = 1.0
ye.loc[ye[0] ==7, 'PermanentCrop'] = 1.0
ye.loc[ye[0] ==8, 'Residential'] = 1.0
ye.loc[ye[0] ==9, 'SeaLake'] = 1.0

ye=ye.drop(ye.columns[0], axis=1)                                    


y=ye.astype('float32')      
print(y)



    
X = np.array(X)#.reshape(-1,img_len, img_len,3)
y = np.array(y)

In [ ]:
print(y.shape,X.shape)
print(np.isnan(y).any())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=196)

In [ ]:
print (X_train)

In [ ]:
input_shape=[64, 64, 3]



model = Sequential() 

model.add(Conv2D(28, (3, 3), padding='same',input_shape=input_shape) 
model.add(Activation('relu')) 
model.add(Conv2D(28, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPool2D(2,2)) 

model.add(Conv2D(56, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(Conv2D(56, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPool2D(2,2))

model.add(Conv2D(112, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(Conv2D(112, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPool2D(2,2)) 

model.add(Flatten()) 
model.add(Dense(784)) 
model.add(Activation('relu')) 
model.add(Dropout(0.6)) 
model.add(Dense(10)) 

model.add(Activation('sigmoid')) 
adam = optimizers.Adam(lr=0.001) 
model.compile(optimizer=adam, loss=losses.binary_crossentropy, metrics=['accuracy'])


          
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=15, 
                    validation_data=(X_test, y_test))


In [ ]:
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
preds2 = model.predict(X_test, verbose=1)

In [ ]:

print(y_test.shape)
y_test= np.array([np.argmax(y) for y in y_test])
y_predict=np.array([ np.argmax(p) for p in preds2])
print(y_test.shape)
print(y_predict.shape)
confusion=confusion_matrix(y_test, y_predict)
sns.heatmap(confusion, annot=True, annot_kws={"size": 14}, fmt='g', linewidths=.5, 
            yticklabels='r''f''a''v''h''i''p''c''re''s',xticklabels='r''f''a''v''h''i''p''c''re''s')
plt.ylabel('True label')
plt.xlabel('Predicted label')